# Refactored code for
* Setting up and running Ollama in Kaggle
* Downloading THUIAR dataset
* Zero-Shot Prompt
* Use LLM to classify intent from an input 'question' dataset
* To configure your file/folder paths, LLM, dataset, start_index and end_index for each run, please update the config.py file

This notebook will also be used as the base to test any fixes to the LLM intent classification pipeline.
* 2025.05.26: Updated results output file from JSON to Pickle, to store list of dictionaries. 1 dictionary contains the results for each record. Lists of dictionaries can be downloaded from multiple notebooks, then concatenated for analysis
* 2025.05.30: Update prompt and bulletpts_intent.
  * Check if dataset contains 'oos' (out of scope) category
  * If dataset has no 'oos' (out of scope) category, turn 1 category into 'oos'. Use updated categories in bulletpts_intent. Also update prompt instructions on when to classify an example as 'oos'
  * **This force_oos fix is implemented in [notebook 01E](https://www.kaggle.com/code/kaiquanmah/01e-kaggle-ollama-llama3-2-w-force-oos?scriptVersionId=242648764)**
* 2025.05.30: Add pydantic schema with enums
  * From an analysis of errors, the model previously had a 45% average accuracy rate across categories. The model predicted a set of categories outside of what we gave it in 'bulletpts_intent'
  * To fix this, we will try to implement a pydantic schema solution for the model to only predict categories from the allowed list of categories ('bulletpts_intent')
* 2025.05.30: Set Ollama chat temperature to 0
  * Previously, we used the default temperature of 0.8, which might have caused the model to predict categories we did not provide to it ([Reading](https://docs.spring.io/spring-ai/reference/api/chat/ollama-chat.html))
  * **The pydantic schema and temperature fixes are implemented in [notebook 01F](https://www.kaggle.com/code/kaiquanmah/01f-kaggle-ollama-llama3-2-w-pydantic-schema)**
* 2025.06.03:
  1. **Remove 'oos' from `bulletpts_intent` input into prompt**, to be consistent with the team's approach when exploring embedding approaches to classify 'oos' examples. **Keep 'oos' in pydantic enums/Literal (for LLM to output 'oos' as an allowed class value)**
  2. **Remove 0.99 when defining the prompt format - to avoid anchoring LLM on outputting confidence of 0.99**
  3. **Added ability for user to define which classes are 'oos'**
  * **These 3 fixes are in [notebook 01G](https://www.kaggle.com/code/kaiquanmah/01g-kaggle-ollama-llama3-2-oos-update)**
* 2025.06.10:
  * From an error analysis earlier, **models can get confused between similar intent classes**
  * Therefore **we will analyse similar intent classes/labels -> get their indexes -> put them into 'oos' in [notebook 01H](https://www.kaggle.com/code/kaiquanmah/01h1-openintent-ollama-llama3-2-3b-banking77)**
  * **Going from zero-shot prompt previously, to few-shot prompt (with 5 examples) from known intents**. These 5 examples were **non-oos, and misclassified previously**. This 'fix' is in **[notebook 01i](https://www.kaggle.com/code/kaiquanmah/01i1-openintent-ollama-llama3-2-3b-banking77)**
* 2025.06.16:
  * For known intents (ie not in the 'oos' class), give 5 examples each in the few-shot prompt **[notebook 01J](https://www.kaggle.com/code/kaiquanmah/01j1-openintent-ollama-llama3-2-3b-banking77)**
* 2025.06.17:
  * Now we explore how changing the number of known intent classes affects the recall of oos in **[notebook 01K](https://www.kaggle.com/code/kaiquanmah/01k1-openintent-ollama-llama3-2-3b-banking77)**
  * For quick experimentation, we implement (1) fewshot prompt with 1 example per known intent class, (2) changing number of known intent classes in various notebook runs, (3) 100 oos sentences for the model to classify (taking from first class for banking77 and stackoverflow dataset, or the oos class for CLINC150 oos dataset)
    * For (3) - Added 'first_class' variable for each dataset to Config
    * For (3) - Created new fn to filter and keep 100 records from 'first/oos class' to input to the model to classify

In [1]:
# 1. create dirs if they do not exist
import os
os.makedirs('/kaggle/working/src', exist_ok=True)
os.makedirs('/kaggle/working/prediction', exist_ok=True)

In [2]:
%%writefile /kaggle/working/src/setup_ollama.py
import os
import subprocess
import time
from src.config import Config # absolute import

# 1. Install Ollama (if not already installed)
try:
    # Check if Ollama is already installed
    subprocess.run(["ollama", "--version"], capture_output=True, check=True)
    print("Ollama is already installed.")
except FileNotFoundError:
    print("Installing Ollama...")
    subprocess.run("curl -fsSL https://ollama.com/install.sh  | sh", shell=True, check=True)

# 2. Start Ollama server in the background
print("Starting Ollama server...")
process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Wait for the server to initialize
time.sleep(5)


# 3. Pull the model
model_name = Config.model_name
print(f"Pulling {model_name} model...")
subprocess.run(["ollama", "pull", model_name], check=True)

# 4. Install Python client
subprocess.run(["pip", "install", "ollama"], check=True)

print("Ollama setup complete!")

Writing /kaggle/working/src/setup_ollama.py


In [3]:
%%writefile requirements.txt
pandas
# numpy
pydantic
typing
# enum
huggingface-hub

Writing requirements.txt


In [4]:
%%writefile /kaggle/working/src/__init__.py
# folder for config

Writing /kaggle/working/src/__init__.py


In [5]:
%%writefile /kaggle/working/src/config.py
class Config:
    target_dir = '/kaggle/working/data' # data directory to clone into
    cloned_data_dir = target_dir + '/data'
    prediction_dir = target_dir + '/prediction'
    dataset_name = 'oos' # UPDATE options: 'banking', 'stackoverflow', 'oos'
    idx2label_target_dir = '/kaggle/working/idx2label'
    idx2label_filename_hf = 'clinc150_oos_idx2label.csv' # UPDATE options: banking77_idx2label.csv, stackoverflow_idx2label.csv, clinc150_oos_idx2label.csv
    fewshot_examples_dir = '/kaggle/working/fewshot'
    fewshot_subdir = '/fewshot-1example-per-nonoos/'
    fewshot_examples_filename = 'oos_14notoos.txt' # UPDATE options: banking_25perc_oos.txt, stackoverflow_25perc_oos.txt, oos_25perc_oos.txt
    list_oos_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136]
    # UPDATE gathered from within the team - for reproducible, comparable results with other open intent classification approaches
    model_name = 'llama3.2:3b'
    start_index=0 # eg: 0, 10001, 11851
    end_index=None # eg: 10, 10000, 11850 or None (use end_index=None to process the full dataset)
    log_every_n_examples=10 # 2
    force_oos = True  # NEW: Add flag to force dataset to contain 'oos' class for the last class value (sorted alphabetically), if 'oos' class does not exist in the original dataset
    filter_oos_qns_only = True
    n_oos_qns = 100
    first_class_banking = 'activate_my_card' # following idx2label
    first_class_stackoverflow = 'wordpress' # following idx2label
    first_class_oos = 'oos'

Writing /kaggle/working/src/config.py


In [6]:
%%writefile download_dataset.py
from src.config import Config
import os
import subprocess
target_dir = Config.target_dir # data directory to clone into
cloned_data_dir = Config.cloned_data_dir

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# do not clone dataset repo if cloned data folder exists
if os.path.exists(cloned_data_dir):
    print("Dataset has already been downloaded. If this is incorrect, please delete the Adaptive-Decision-Boundary 'data' folder.")
else:
    # Clone the repository
    subprocess.run(["git",
                    "clone",
                    "https://github.com/thuiar/Adaptive-Decision-Boundary.git",
                    target_dir
                   ])

Writing download_dataset.py


In [7]:
%%writefile predict_class.py
from src.config import Config
import pandas as pd
import os
import ollama
import json
import pickle
import time
from pydantic import BaseModel
from typing import Literal
# from enum import Enum
from huggingface_hub import snapshot_download



# Config.target_dir
# Config.cloned_data_dir'
# Config.dataset_name
# Config.model_name
# Config.start_index
# Config.end_index
# Config.log_every_n_examples


#######################
# load data
#######################
def load_data(data_dir):
    """Loads train, dev, and test datasets from a specified directory."""

    main_df = pd.DataFrame()
    for split in ['train', 'dev', 'test']:
        file_path = os.path.join(data_dir, f'{split}.tsv')
        if os.path.exists(file_path):
          try:
            df = pd.read_csv(file_path, sep='\t')
            df['dataset'] = os.path.basename(data_dir)
            df['split'] = split
            main_df = pd.concat([main_df, df], ignore_index=True)
          except pd.errors.ParserError as e:
            print(f"Error parsing {file_path}: {e}")
            # Handle the error appropriately, e.g., skip the file, log the error, etc.
        else:
            print(f"Warning: {split}.tsv not found in {data_dir}")
    return main_df


def filter100examples_oos(dataset_name, df):
    # dont input 'only oos qns to model'
    if Config.filter_oos_qns_only == False:
        filtered_df = df
    # vs
    # input 'only oos qns to model'
    else:
        if dataset_name == 'banking':
            first_class = Config.first_class_banking
        elif dataset_name == 'stackoverflow':
            first_class = Config.first_class_stackoverflow
        else:
            first_class = Config.first_class_oos
    
        filtered_df = df.copy()
        filtered_df = filtered_df.loc[filtered_df["label"] == first_class]
        filtered_df = filtered_df.sample(n=Config.n_oos_qns, random_state=38)
    return filtered_df


df = pd.DataFrame()

data_dir = os.path.join(Config.cloned_data_dir, Config.dataset_name)
if os.path.exists(data_dir):
  df = load_data(data_dir)
  print(f"Loaded dataset into dataframe: {Config.dataset_name}")
  print(f"Dimensions: {df.shape}")
  print(f"Col names: {df.columns}")
else:
  print(f"Warning: Directory {data_dir} not found.")
#######################



#######################
# unique intents
#######################
sorted_intent = list(sorted(df.label.unique()))
print("="*80)
print(f"Original dataset intents: {sorted_intent}")
print(f"Number of original intents: {len(sorted_intent)}\n")


# 2025.06.03
# New OOS approach - get 25/50/75% of class indexes for each dataset within the team (for reproducibility and comparable results)
# Change their class labels to 'oos'
snapshot_download(repo_id="KaiquanMah/open-intent-query-classification", repo_type="space", allow_patterns="*_idx2label.csv", local_dir=Config.idx2label_target_dir)
idx2label_filepath = Config.idx2label_target_dir + '/dataset_idx2label/' + Config.idx2label_filename_hf
idx2label = pd.read_csv(idx2label_filepath)
idx2label_oos = idx2label[idx2label.index.isin(Config.list_oos_idx)]
idx2label_oos.reset_index(drop=True, inplace=True)

# 2025.06.17 keep track of non-oos labels, to use in IntentSchema
nonoos_labels = idx2label[~idx2label.label.isin(Config.list_oos_idx)]['label'].values
print("="*80)
print("Original intents to convert to OOS class")
print(idx2label_oos)
print(f"Percentage of original intents to convert to OOS class: {len(idx2label_oos)/len(idx2label)}\n")

oos_labels = idx2label_oos['label'].values
list_sorted_intent_aft_conversion = ['oos' if intent.lower() in oos_labels else intent for intent in sorted_intent]
list_sorted_intent_aft_conversion_deduped = sorted(set(list_sorted_intent_aft_conversion))
print("="*80)
print("Unique intents after converting some to OOS class")
print(list_sorted_intent_aft_conversion_deduped)
print(f"Number of unique intents after converting some to OOS class: {len(list_sorted_intent_aft_conversion_deduped)}\n")



# unique intents - from set to bullet points (to use in prompts)
# bulletpts_intent = "\n".join(f"- {category}" for category in set_intent)
# 2025.06.03: do not show 'oos' in the prompt (to avoid leakage of 'oos' class)
bulletpts_intent = "\n".join(f"- {category}" for category in list_sorted_intent_aft_conversion_deduped if category and (category!='oos'))

# 2025.06.04: fix adjustment if 'oos' is already in the original dataset
int_oos_in_orig_dataset = int('oos' in idx2label.label.values)
adjust_if_oos_not_in_orig_dataset = [0 if int_oos_in_orig_dataset == 1 else 1][0]

print("="*80)
print("sanity check")
print(f"Number of original intents: {len(sorted_intent)}")
print(f"Number of original intents + 1 OOS class (if doesnt exist in original dataset): {len(sorted_intent) + adjust_if_oos_not_in_orig_dataset}")
print(f"Number of original intents to convert to OOS class: {len(idx2label_oos)}")
print(f"Percentage of original intents to convert to OOS class: {len(idx2label_oos)/len(idx2label)}")
print(f"Number of unique intents after converting some to OOS class: {len(list_sorted_intent_aft_conversion_deduped)}")
print(f"Number of original intents + 1 OOS class (if doesnt exist in original dataset) - converted classes: {len(sorted_intent) + adjust_if_oos_not_in_orig_dataset - len(idx2label_oos)}")
print(f"Numbers match: {(len(sorted_intent) + adjust_if_oos_not_in_orig_dataset - len(idx2label_oos)) == len(list_sorted_intent_aft_conversion_deduped)}")
print("Prepared unique intents")
#######################




#######################
# Enforce schema on the model (e.g. allowed list of predicted categories)
#######################

class IntentSchema(BaseModel):
    # dynamically unpack list of categories for different dataset(s)
    category: Literal[*list_sorted_intent_aft_conversion_deduped]
    confidence: float
    
#######################




#######################
# filter after preparing intents
#######################
df = filter100examples_oos(Config.dataset_name, df)
print("Filtered dataset")
print(f"Dimensions: {df.shape}")
print(f"Col names: {df.columns}")
#######################



#######################
# Prompt
#######################
# prompt 2 with less information/compute, improve efficiency
# 2025.06.10 prompt 3 with 5 few shot examples only - notebook O1H1, O1i1
# 2025.06.16 prompt 4 with 5 examples per each known intent (ie non-oos intent) - notebook 01J1
snapshot_download(repo_id="KaiquanMah/open-intent-query-classification", repo_type="space", allow_patterns="*.txt", local_dir=Config.fewshot_examples_dir)
with open(Config.fewshot_examples_dir + Config.fewshot_subdir + Config.fewshot_examples_filename, 'r') as file:
    fewshot_examples = file.read()

def get_prompt(dataset_name, split, question, categories, fewshot_examples):
    
    prompt = f'''
You are an expert in understanding and identifying what users are asking you.

Your task is to analyze an input query from a user and assign the most appropriate category from the following list:
{categories}

Only classify as "oos" (out of scope category) if none of the other categories apply.

Below are several examples to guide your classification:

---
{fewshot_examples}
---

===============================

New Question: {question}

===============================

Provide your final classification in **valid JSON format** with the following structure:
{{
  "category": "your_chosen_category_name",
  "confidence": confidence_level_rounded_to_the_nearest_2_decimal_places
}}


Ensure the JSON has:
- Opening and closing curly braces
- Double quotes around keys and string values
- Confidence as a number (not a string), with maximum 2 decimal places

Do not include any explanations or extra text.
            '''
    return prompt



#######################


#######################
# Model on 1 Dataset
#######################
# Save a list of dictionaries 
# containing a dictionary for each record's
# - predicted category
# - confidence level and
# - original dataframe values

def predict_intent(model_name, df, categories, start_index=0, end_index=None, log_every_n_examples=100):
    start_time = time.time()
    results = []  # Store processed results
    
    # Slice DataFrame based on start/end indices
    if end_index is None:
        subset_df = df.iloc[start_index:]
    else:
        subset_df = df.iloc[start_index:end_index+1]
    
    total_rows = len(subset_df)
    subset_row_count = 0
    
    
    for row in subset_df.itertuples():
        subset_row_count+=1
        prompt = get_prompt(row.dataset, row.split, row.text, categories, fewshot_examples)
        if subset_row_count == 1:
            print("Example of how prompt looks, for the 1st example in this subset of data")
            print(prompt)

            print("Example of how IntentSchema looks")
            print(IntentSchema.model_json_schema())
        
        
        try:
            response = ollama.chat(model=model_name, 
                                   messages=[
                                                {'role': 'user', 'content': prompt}
                                            ],
                                   format = IntentSchema.model_json_schema(),
                                   options = {'temperature': 0},  # Set temperature to 0 for a more deterministic output
                                  )
            msg = response['message']['content']
            parsed = json.loads(msg)

            
            # Safely extract keys with defaults - resolve parsing error
            # maybe LLM did not output a particular key-value pair
            category = parsed.get('category', 'error')
            confidence = parsed.get('confidence', 0.0)
            parsed = {'category': category, 'confidence': confidence}
        except (json.JSONDecodeError, KeyError, Exception) as e:
            parsed = {'category': 'error', 'confidence': 0.0}
        
        # Combine original row data with predictions
        results.append({
            "Index": row.Index,
            "text": row.text,
            "label": row.label,
            "dataset": row.dataset,
            "split": row.split,
            "predicted": parsed['category'],
            "confidence": parsed['confidence']
        })

        
        # Log progress
        if subset_row_count % log_every_n_examples == 0:
            elapsed_time = time.time() - start_time
            
            avg_time_per_row = elapsed_time / subset_row_count
            remaining_rows = total_rows - subset_row_count
            eta = avg_time_per_row * remaining_rows
            
            print(f"Processed original df idx {row.Index} (subset row {subset_row_count}) | "
                  f"Elapsed: {elapsed_time:.2f}s | ETA: {eta:.2f}s")
    
    return results  # Return list of dictionaries
    

print(f"Starting intent classification using {Config.model_name}")
subset_results = predict_intent(Config.model_name, 
                                df, 
                                bulletpts_intent, 
                                start_index = Config.start_index, 
                                end_index = Config.end_index,
                                log_every_n_examples = Config.log_every_n_examples)



# update end_index for filename (if None is used for the end of the df)
# Get the last index of the DataFrame
last_index = df.index[-1] 
# Use last index if Config.end_index is None
end_index = Config.end_index if Config.end_index is not None else last_index



# 2025.05.23 changed from JSON to PKL
# because we are saving list of dictionaries
# Save to PKL
# 2025.06.04 explore changing back to JSON
# with open(f'results_{Config.model_name}_{Config.dataset_name}_{Config.start_index}_{end_index}.pkl', 'wb') as f:
#     pickle.dump(subset_results, f)
with open(f'results_{Config.model_name}_{Config.dataset_name}_{Config.start_index}_{end_index}.json', 'w') as f:
    json.dump(subset_results, f, indent=2)

print("Completed intent classification")


#######################


Writing predict_class.py


In [8]:
%%writefile /kaggle/working/main.py
import subprocess
import sys


# 1. Install libraries from requirements.txt
print("Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "/kaggle/working/requirements.txt"], check=True)

# 2. Run setup_ollama.py
print("Starting Ollama setup...")
# subprocess.run(["python3", "/kaggle/working/src/setup_ollama.py"], check=True)
print("Starting Ollama setup...")
subprocess.run(
    ["python3", "-m", "src.setup_ollama"],  # Run as a module
    cwd="/kaggle/working",  # Set working directory to parent of 'src'
    check=True
)

# 3. Run download_dataset.py
print("Downloading dataset...")
subprocess.run(["python3", "/kaggle/working/download_dataset.py"], check=True)

# 4. Run predict_class.py
print("Running prediction script...")
subprocess.run(["python3", "/kaggle/working/predict_class.py"], check=True)

Writing /kaggle/working/main.py


# Model on subset of examples

In [9]:
!python3 /kaggle/working/main.py

Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=cda700cdb90210a21587fa03337757998fd3ee91ba0f021e00ae392d20ad8dd0
  Stored in directory: /root/.cache/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built typing
Starting Ollama setup...
Starting Ollama setup...
Installing Ollama...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Starting Ollama server...
Pulling llama3.2:3b model...
pull

# Sanity check folders

In [10]:
!cd /kaggle/working/ && ls -la

total 408
drwxr-xr-x 7 root root   4096 Jun 18 14:38 .
drwxr-xr-x 8 root root   4096 Jun 18 14:11 ..
drwxr-xr-x 7 root root   4096 Jun 18 14:13 data
-rw-r--r-- 1 root root    695 Jun 18 14:11 download_dataset.py
drwxr-xr-x 7 root root   4096 Jun 18 14:13 fewshot
drwxr-xr-x 4 root root   4096 Jun 18 14:13 idx2label
-rw-r--r-- 1 root root    846 Jun 18 14:11 main.py
---------- 1 root root 343701 Jun 18 14:38 __notebook__.ipynb
-rw-r--r-- 1 root root  12013 Jun 18 14:11 predict_class.py
drwxr-xr-x 2 root root   4096 Jun 18 14:11 prediction
-rw-r--r-- 1 root root     54 Jun 18 14:11 requirements.txt
-rw-r--r-- 1 root root  20178 Jun 18 14:38 results_llama3.2:3b_oos_0_23401.json
drwxr-xr-x 3 root root   4096 Jun 18 14:11 src


In [11]:
!cd /kaggle/working/src && ls -la

total 24
drwxr-xr-x 3 root root 4096 Jun 18 14:11 .
drwxr-xr-x 7 root root 4096 Jun 18 14:38 ..
-rw-r--r-- 1 root root 2034 Jun 18 14:11 config.py
-rw-r--r-- 1 root root   20 Jun 18 14:11 __init__.py
drwxr-xr-x 2 root root 4096 Jun 18 14:11 __pycache__
-rw-r--r-- 1 root root  965 Jun 18 14:11 setup_ollama.py


In [12]:
!cd /kaggle/working/data/data && ls -la

total 20
drwxr-xr-x 5 root root 4096 Jun 18 14:13 .
drwxr-xr-x 7 root root 4096 Jun 18 14:13 ..
drwxr-xr-x 2 root root 4096 Jun 18 14:13 banking
drwxr-xr-x 2 root root 4096 Jun 18 14:13 oos
drwxr-xr-x 2 root root 4096 Jun 18 14:13 stackoverflow


# idx2label_oos examples

In [13]:
pip install huggingface-hub

Note: you may need to restart the kernel to use updated packages.


In [14]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="KaiquanMah/open-intent-query-classification", repo_type="space", allow_patterns="*_idx2label.csv", local_dir='/kaggle/working/idx2label')

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

'/kaggle/working/idx2label'

In [15]:
import pandas as pd
idx2label = pd.read_csv('/kaggle/working/idx2label/dataset_idx2label/banking77_idx2label.csv')
idx2label

,index,label
0,0,activate_my_card
1,1,age_limit
2,2,apple_pay_or_google_pay
3,3,atm_support
4,4,automatic_top_up
...,...,...
72,72,virtual_card_not_working
73,73,visa_or_mastercard
74,74,why_verify_identity
75,75,wrong_amount_of_cash_received


In [16]:
idx2label_oos = idx2label[idx2label.index.isin([31,32,33,36])]
idx2label_oos

,index,label
31,31,exchange_charge
32,32,exchange_rate
33,33,exchange_via_app
36,36,fiat_currency_support


In [17]:
print(idx2label_oos)

    index                  label
31     31        exchange_charge
32     32          exchange_rate
33     33       exchange_via_app
36     36  fiat_currency_support


In [18]:
idx2label_oos.shape

(4, 2)

In [19]:
# percentage of OOS classes over ALL classes in the dataset
len(idx2label_oos)/len(idx2label)

0.05194805194805195